## Cas pratique : 🦠 COVID-19 - Suivi des cas actifs

## Input

### Import des librairies

In [1]:
import pandas as pd
from os import path
import markdown2
import naas
from datetime import datetime, timedelta

### Initialisation du fichier référentiel

In [2]:
ref_path = "REF_CS.xlsx"
sheet_name = "REF_CONTINENT"

if path.exists(ref_path):
    ref_cs = pd.read_excel(ref_path, sheet_name=sheet_name)
else:
    raise ValueError(f"Le fichier {ref_path} n'éxiste pas.")

ref_cs

,COUNTRY,CONTINENT
0,Afghanistan,Asia
1,Albania,Europe
2,Algeria,Africa
3,Andorra,Europe
4,Angola,Africa
...,...,...
191,Lesotho,Africa
192,Marshall Islands,Oceania
193,Solomon Islands,Oceania
194,Vanuatu,Oceania


### Récupérez vos données sources

In [3]:
# Url racine des fichiers sur Github
url = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
       "csse_covid_19_data/csse_covid_19_time_series/")

# Liste des différents fichiers
list_csv = ["time_series_covid19_confirmed_global.csv",
            "time_series_covid19_deaths_global.csv",
            "time_series_covid19_recovered_global.csv"]

# Tableau ou dataframe initial vide
db_init = pd.DataFrame()

# Boucle pour itérer sur les noms de fichiers
for csv in list_csv:
    url_csv = f"{url}{csv}"
    df = pd.read_csv(url_csv)
    df['TYPE'] = csv

    # Concaténation du tableau dans le dataframe initial db_init
    db_init = pd.concat([df, db_init], axis=0)

db_init.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21,10/4/21,10/5/21,TYPE
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,time_series_covid19_recovered_global.csv
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,time_series_covid19_recovered_global.csv
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,time_series_covid19_recovered_global.csv
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,time_series_covid19_recovered_global.csv
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,time_series_covid19_recovered_global.csv


## Model

### Nettoyage des données

In [4]:
db_clean = db_init.copy()

# Selectionner les colonnes
to_drop = ['Province/State', 'Lat', 'Long']
db_clean = db_clean.drop(to_drop, axis=1)

# Renommer les colonnes
to_rename = {'Country/Region': 'COUNTRY'}
db_clean = db_clean.rename(columns=to_rename)

# Selectionner une suite de caractère dans une valeur de type string
db_clean['TYPE'] = db_clean['TYPE'].str[20:-11]

# Mettre en majuscule une valeur de type string
db_clean['TYPE'] = db_clean['TYPE'].str.capitalize()
db_clean.head(5)

,COUNTRY,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21,10/4/21,10/5/21,TYPE
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Recovered
1,Albania,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Recovered
2,Algeria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Recovered
3,Andorra,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Recovered
4,Angola,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Recovered


### Mettre au format base de donnée

In [5]:
db_pivot = db_clean.copy()

# Pivot des colonnes de date en ligne
to_keep = ["COUNTRY", "TYPE"]
db_pivot = pd.melt(db_pivot,
                   id_vars=to_keep,
                   var_name="DATE",
                   value_name="VALUE")

# Agrégation des lignes
to_group = ["COUNTRY", "TYPE", "DATE"]
to_agg = {"VALUE": "sum"}
db_group = db_pivot.groupby(to_group, as_index=False).agg(to_agg)

# Mise au format de la date
db_group["DATE"] = pd.to_datetime(db_group["DATE"], format="%m/%d/%y")
db_group.head(5)

,COUNTRY,TYPE,DATE,VALUE
0,Afghanistan,Confirmed,2021-01-01,52513
1,Afghanistan,Confirmed,2021-01-10,53489
2,Afghanistan,Confirmed,2021-01-11,53538
3,Afghanistan,Confirmed,2021-01-12,53584
4,Afghanistan,Confirmed,2021-01-13,53690


### Enrichissez vos données avec des calculs, référentiels et mappings

In [6]:
db_enr = db_group.copy()

# Jointure avec le référentiel
db_enr = pd.merge(db_enr, ref_cs, on="COUNTRY", how="left")

# Calcul des cas actifs
db_ac = db_enr.copy()
db_ac.loc[db_ac["TYPE"] == "Deaths", "VALUE"] = db_ac["VALUE"] * (-1)
db_ac.loc[db_ac["TYPE"] == "Recovered", "VALUE"] = db_ac["VALUE"] * (-1)
db_ac["TYPE"] = "Active Cases"
to_group = ["COUNTRY", "TYPE", "DATE", "CONTINENT"]
to_agg = {"VALUE": "sum"}
db_ac = db_ac.groupby(to_group, as_index=False).agg(to_agg)

# Concaténation des cas actifs avec le dataframe principal
db_bdd = pd.concat([db_ac, db_enr], axis=0)

# Règle conditionnelle pour identifier la date max
date_max = db_bdd["DATE"].max()
db_bdd["LAST_DATE"] = db_bdd.apply(lambda row: 1 if row["DATE"] == date_max else 0, axis=1)
db_bdd.head(5)

,COUNTRY,TYPE,DATE,CONTINENT,VALUE,LAST_DATE
0,Afghanistan,Active Cases,2020-01-22,Asia,0,0
1,Afghanistan,Active Cases,2020-01-23,Asia,0,0
2,Afghanistan,Active Cases,2020-01-24,Asia,0,0
3,Afghanistan,Active Cases,2020-01-25,Asia,0,0
4,Afghanistan,Active Cases,2020-01-26,Asia,0,0


### Sauvegardez vos données

In [7]:
# Date courante
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Ajout de la date courante au dataframe
db_bdd["DATE_MAJ"] = now

# Sauvegarder le dataframe en csv
output_path = "BASE_COVID.csv"
db_bdd.to_csv(output_path, sep=";", decimal=".", index=False)

# Afficher le message de succès de la sauvegarde
to_print = f"Votre fichier '{output_path}' a bien été sauvegardé !"
print(to_print)
db_bdd

Votre fichier 'BASE_COVID.csv' a bien été sauvegardé !


,COUNTRY,TYPE,DATE,CONTINENT,VALUE,LAST_DATE,DATE_MAJ
0,Afghanistan,Active Cases,2020-01-22,Asia,0,0,2021-10-06 23:43:25
1,Afghanistan,Active Cases,2020-01-23,Asia,0,0,2021-10-06 23:43:25
2,Afghanistan,Active Cases,2020-01-24,Asia,0,0,2021-10-06 23:43:25
3,Afghanistan,Active Cases,2020-01-25,Asia,0,0,2021-10-06 23:43:25
4,Afghanistan,Active Cases,2020-01-26,Asia,0,0,2021-10-06 23:43:25
...,...,...,...,...,...,...,...
364450,Zimbabwe,Recovered,2021-09-07,Africa,0,0,2021-10-06 23:43:25
364451,Zimbabwe,Recovered,2020-09-08,Africa,5477,0,2021-10-06 23:43:25
364452,Zimbabwe,Recovered,2021-09-08,Africa,0,0,2021-10-06 23:43:25
364453,Zimbabwe,Recovered,2020-09-09,Africa,5542,0,2021-10-06 23:43:25


In [8]:
naas.asset.add(output_path)

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


'https://public.naas.ai/amVyZW15LTJFcmF2ZW5lbC00MGNhc2hzdG9yeS0yRWNvbQ==/asset/481eb67aec50b77a847d2c6c4dc6a986934648c9c2550a0d7d2942b4213b'

### Récupération des variables à intégrer dans le mail : Données COVID-19

In [9]:
covid_path = "BASE_COVID.csv"
#covid_path = path.join("..", "output", covid_csv)
db_covid_init = pd.read_csv(covid_path, sep=";", decimal=".")

db_covid_init.head(5)

,COUNTRY,TYPE,DATE,CONTINENT,VALUE,LAST_DATE,DATE_MAJ
0,Afghanistan,Active Cases,2020-01-22,Asia,0,0,2021-10-06 23:43:25
1,Afghanistan,Active Cases,2020-01-23,Asia,0,0,2021-10-06 23:43:25
2,Afghanistan,Active Cases,2020-01-24,Asia,0,0,2021-10-06 23:43:25
3,Afghanistan,Active Cases,2020-01-25,Asia,0,0,2021-10-06 23:43:25
4,Afghanistan,Active Cases,2020-01-26,Asia,0,0,2021-10-06 23:43:25


### Enrichissement : Calcul des variations pour récupérer les évolutions quotidiennes les plus importants

In [10]:
db_covid = db_covid_init.copy()

# Format de la colonne DATE
db_covid['DATE'] = pd.to_datetime(db_covid['DATE'])

# Ajout de la date de comparaison dans le data frame principal
db_covid['DATE_COMP'] = db_covid['DATE'] + timedelta(days=-1)

db_covid

,COUNTRY,TYPE,DATE,CONTINENT,VALUE,LAST_DATE,DATE_MAJ,DATE_COMP
0,Afghanistan,Active Cases,2020-01-22,Asia,0,0,2021-10-06 23:43:25,2020-01-21
1,Afghanistan,Active Cases,2020-01-23,Asia,0,0,2021-10-06 23:43:25,2020-01-22
2,Afghanistan,Active Cases,2020-01-24,Asia,0,0,2021-10-06 23:43:25,2020-01-23
3,Afghanistan,Active Cases,2020-01-25,Asia,0,0,2021-10-06 23:43:25,2020-01-24
4,Afghanistan,Active Cases,2020-01-26,Asia,0,0,2021-10-06 23:43:25,2020-01-25
...,...,...,...,...,...,...,...,...
483443,Zimbabwe,Recovered,2021-09-07,Africa,0,0,2021-10-06 23:43:25,2021-09-06
483444,Zimbabwe,Recovered,2020-09-08,Africa,5477,0,2021-10-06 23:43:25,2020-09-07
483445,Zimbabwe,Recovered,2021-09-08,Africa,0,0,2021-10-06 23:43:25,2021-09-07
483446,Zimbabwe,Recovered,2020-09-09,Africa,5542,0,2021-10-06 23:43:25,2020-09-08


In [11]:
db_value_last = db_covid.copy()

# Création du dataframe pour récupérer la valeur précédente
to_delete = ['CONTINENT', 'LAST_DATE', 'DATE_MAJ', "DATE_COMP"]
db_value_last = db_covid.drop(to_delete, axis=1)

to_rename = {'DATE': 'DATE_COMP',
             'VALUE': 'VALUE_COMP'}
db_value_last = db_value_last.rename(columns=to_rename)

db_value_last

,COUNTRY,TYPE,DATE_COMP,VALUE_COMP
0,Afghanistan,Active Cases,2020-01-22,0
1,Afghanistan,Active Cases,2020-01-23,0
2,Afghanistan,Active Cases,2020-01-24,0
3,Afghanistan,Active Cases,2020-01-25,0
4,Afghanistan,Active Cases,2020-01-26,0
...,...,...,...,...
483443,Zimbabwe,Recovered,2021-09-07,0
483444,Zimbabwe,Recovered,2020-09-08,5477
483445,Zimbabwe,Recovered,2021-09-08,0
483446,Zimbabwe,Recovered,2020-09-09,5542


In [12]:
# Jointure entre les 2 tables
join_on = ['COUNTRY', 'TYPE', 'DATE_COMP']
db_covid = pd.merge(db_covid,
                    db_value_last,
                    on=join_on,
                    how='left')

db_covid

,COUNTRY,TYPE,DATE,CONTINENT,VALUE,LAST_DATE,DATE_MAJ,DATE_COMP,VALUE_COMP
0,Afghanistan,Active Cases,2020-01-22,Asia,0,0,2021-10-06 23:43:25,2020-01-21,NaN
1,Afghanistan,Active Cases,2020-01-23,Asia,0,0,2021-10-06 23:43:25,2020-01-22,0.0
2,Afghanistan,Active Cases,2020-01-24,Asia,0,0,2021-10-06 23:43:25,2020-01-23,0.0
3,Afghanistan,Active Cases,2020-01-25,Asia,0,0,2021-10-06 23:43:25,2020-01-24,0.0
4,Afghanistan,Active Cases,2020-01-26,Asia,0,0,2021-10-06 23:43:25,2020-01-25,0.0
...,...,...,...,...,...,...,...,...,...
483443,Zimbabwe,Recovered,2021-09-07,Africa,0,0,2021-10-06 23:43:25,2021-09-06,0.0
483444,Zimbabwe,Recovered,2020-09-08,Africa,5477,0,2021-10-06 23:43:25,2020-09-07,5455.0
483445,Zimbabwe,Recovered,2021-09-08,Africa,0,0,2021-10-06 23:43:25,2021-09-07,0.0
483446,Zimbabwe,Recovered,2020-09-09,Africa,5542,0,2021-10-06 23:43:25,2020-09-08,5477.0


In [13]:
db_covid_final = db_covid.copy()

db_covid_final = db_covid_final.drop(['DATE_COMP'], axis=1)

db_covid_final["VARV"] = db_covid_final["VALUE"] - db_covid_final["VALUE_COMP"]
db_covid_final

,COUNTRY,TYPE,DATE,CONTINENT,VALUE,LAST_DATE,DATE_MAJ,VALUE_COMP,VARV
0,Afghanistan,Active Cases,2020-01-22,Asia,0,0,2021-10-06 23:43:25,NaN,NaN
1,Afghanistan,Active Cases,2020-01-23,Asia,0,0,2021-10-06 23:43:25,0.0,0.0
2,Afghanistan,Active Cases,2020-01-24,Asia,0,0,2021-10-06 23:43:25,0.0,0.0
3,Afghanistan,Active Cases,2020-01-25,Asia,0,0,2021-10-06 23:43:25,0.0,0.0
4,Afghanistan,Active Cases,2020-01-26,Asia,0,0,2021-10-06 23:43:25,0.0,0.0
...,...,...,...,...,...,...,...,...,...
483443,Zimbabwe,Recovered,2021-09-07,Africa,0,0,2021-10-06 23:43:25,0.0,0.0
483444,Zimbabwe,Recovered,2020-09-08,Africa,5477,0,2021-10-06 23:43:25,5455.0,22.0
483445,Zimbabwe,Recovered,2021-09-08,Africa,0,0,2021-10-06 23:43:25,0.0,0.0
483446,Zimbabwe,Recovered,2020-09-09,Africa,5542,0,2021-10-06 23:43:25,5477.0,65.0


### Récupération des données des cas actifs

In [14]:
#Récupération des données des cas actifs
db_active = db_covid_final.copy()

# Filtre sur les données
db_active = db_active[(db_active["LAST_DATE"] == 1) & 
                      (db_active["TYPE"] == "Active Cases")].reset_index(drop=True)
db_active

,COUNTRY,TYPE,DATE,CONTINENT,VALUE,LAST_DATE,DATE_MAJ,VALUE_COMP,VARV
0,Afghanistan,Active Cases,2021-10-05,Asia,148095,1,2021-10-06 23:43:25,148075.0,20.0
1,Albania,Active Cases,2021-10-05,Europe,169893,1,2021-10-06 23:43:25,169081.0,812.0
2,Algeria,Active Cases,2021-10-05,Africa,198215,1,2021-10-06 23:43:25,198089.0,126.0
3,Andorra,Active Cases,2021-10-05,Europe,15141,1,2021-10-06 23:43:25,15137.0,4.0
4,Angola,Active Cases,2021-10-05,Africa,58308,1,2021-10-06 23:43:25,57366.0,942.0
...,...,...,...,...,...,...,...,...,...
186,Vietnam,Active Cases,2021-10-05,Asia,798345,1,2021-10-06 23:43:25,794116.0,4229.0
187,West Bank and Gaza,Active Cases,2021-10-05,Asia,405319,1,2021-10-06 23:43:25,404055.0,1264.0
188,Yemen,Active Cases,2021-10-05,Asia,7486,1,2021-10-06 23:43:25,7471.0,15.0
189,Zambia,Active Cases,2021-10-05,Africa,205549,1,2021-10-06 23:43:25,205523.0,26.0


In [15]:
# Dernière date de mise à jour des données + date d'extraction
maj_covid = db_active.loc[0, "DATE"].strftime("%d/%m/%Y")
print(maj_covid)

05/10/2021


In [16]:
db_active_plus = db_active.copy()
db_active_plus = db_active_plus.nlargest(5, "VARV").reset_index(drop=True)
db_active_plus

,COUNTRY,TYPE,DATE,CONTINENT,VALUE,LAST_DATE,DATE_MAJ,VALUE_COMP,VARV
0,US,Active Cases,2021-10-05,North America,43242099,1,2021-10-06 23:43:25,43148760.0,93339.0
1,United Kingdom,Active Cases,2021-10-05,Europe,7867958,1,2021-10-06 23:43:25,7834934.0,33024.0
2,Turkey,Active Cases,2021-10-05,Asia,7231712,1,2021-10-06 23:43:25,7202138.0,29574.0
3,Russia,Active Cases,2021-10-05,Europe,7316533,1,2021-10-06 23:43:25,7292944.0,23589.0
4,Brazil,Active Cases,2021-10-05,South America,20900245,1,2021-10-06 23:43:25,20880394.0,19851.0


In [17]:
db_active_less = db_active.copy()
db_active_less = db_active_less.nsmallest(5, "VARV").reset_index(drop=True)
db_active_less

,COUNTRY,TYPE,DATE,CONTINENT,VALUE,LAST_DATE,DATE_MAJ,VALUE_COMP,VARV
0,El Salvador,Active Cases,2021-10-05,North America,101036,1,2021-10-06 23:43:25,101055.0,-19.0
1,Mauritius,Active Cases,2021-10-05,Africa,15676,1,2021-10-06 23:43:25,15686.0,-10.0
2,Bahamas,Active Cases,2021-10-05,North America,20755,1,2021-10-06 23:43:25,20755.0,0.0
3,Benin,Active Cases,2021-10-05,Africa,23731,1,2021-10-06 23:43:25,23731.0,0.0
4,Cameroon,Active Cases,2021-10-05,Africa,90844,1,2021-10-06 23:43:25,90844.0,0.0


### Automation

In [18]:
# Envoi des éléments en production
naas.dependency.add(ref_path)
naas.scheduler.add(recurrence="30 12 * * *")

👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
recurrence is deprecated use cron arg instead
👌 Well done! Your Notebook has been sent to production. 

⏰ It will be scheduled "At 12:30 every day" (more on the syntax on https://crontab.guru/).

Ps: to remove the "Scheduler", just replace .add by .delete


In [19]:
# import naas
# naas.scheduler.delete()

## Output

### Paramètres de l'email

In [20]:
email = ["jeremy.ravenel@cashstory.com"]
now = datetime.now().strftime("%d/%m/%Y %H:%M")
subject = f"🦠 COVID-19 - Suivi des cas actifs: {now}"
excel_report = "Dashboard-Excel.xlsx"

### Création du texte variable dans l'email

In [21]:
def create_covid_text(df):
    string = ""
    
    for _, row in df.iterrows():
        country = row["COUNTRY"]
        value = row["VALUE"]
        varv = row["VARV"]
        order = _+1
        
        # Format value data
        value = '{:,.0f}'.format(value).replace(",", " ")
        varv = '{:+,.0f}'.format(varv).replace(",", " ")
        
        tmp_string = f"<li>{order}. {country}: {varv} ({value})</li>"
        string = f"{string}{tmp_string}"
    return string

most_active = create_covid_text(db_active_plus)
print(most_active)

less_active = create_covid_text(db_active_less)
print(less_active)

<li>1. US: +93 339 (43 242 099)</li><li>2. United Kingdom: +33 024 (7 867 958)</li><li>3. Turkey: +29 574 (7 231 712)</li><li>4. Russia: +23 589 (7 316 533)</li><li>5. Brazil: +19 851 (20 900 245)</li>
<li>1. El Salvador: -19 (101 036)</li><li>2. Mauritius: -10 (15 676)</li><li>3. Bahamas: +0 (20 755)</li><li>4. Benin: +0 (23 731)</li><li>5. Cameroon: +0 (90 844)</li>


In [22]:
# Exposition du fichier Excel et récupération du lien url
excel_url = naas.asset.add(excel_report)
print(excel_url)

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete
https://public.naas.ai/amVyZW15LTJFcmF2ZW5lbC00MGNhc2hzdG9yeS0yRWNvbQ==/asset/b52cc9688335337de935df8232f95366e6541c129b3e9a83e0bf173e51d1


### Remplacement des variables dans le markdown

In [23]:
# Nom et chemin du fichier
markdown_path = "Email_template_COVID19.md"
#markdown_path = path.join("..", "input", markdown_name)

# Lecture du markdown
content = open(markdown_path).read()
html = markdown2.markdown(content)

# Remplacer les variables
html = html.replace("{maj_covid}", maj_covid)
html = html.replace("{most_active}", most_active)
html = html.replace("{less_active}", less_active)
html = html.replace("{excel_report}", excel_url)
print(html)

<p>Bonjour,</p>

<p>Voici ton brief quotidien sur la situation du monde.</p>

<hr />

<h3>🏥 COVID-19 - Suivi des cas actifs <br></h3>

<p>Mise à jour: 05/10/2021</p>

<p><u>Les plus fortes augmentations :</u></p>

<p><li>1. US: +93 339 (43 242 099)</li><li>2. United Kingdom: +33 024 (7 867 958)</li><li>3. Turkey: +29 574 (7 231 712)</li><li>4. Russia: +23 589 (7 316 533)</li><li>5. Brazil: +19 851 (20 900 245)</li></p>

<p><u>Les plus fortes diminutions :</u></p>

<p><li>1. El Salvador: -19 (101 036)</li><li>2. Mauritius: -10 (15 676)</li><li>3. Bahamas: +0 (20 755)</li><li>4. Benin: +0 (23 731)</li><li>5. Cameroon: +0 (90 844)</li></p>

<p>Pour aller plus loin, téléchargez le <a href="https://public.naas.ai/amVyZW15LTJFcmF2ZW5lbC00MGNhc2hzdG9yeS0yRWNvbQ==/asset/b52cc9688335337de935df8232f95366e6541c129b3e9a83e0bf173e51d1">tableau bord COVID-19</a>.</p>

<p><em>PS: Une fois le fichier ouvert, n'oubliez pas d'actualiser les données.</em></p>

<hr />

<p><strong><u>À propos de ces donnée

### Envoi du mail

In [24]:
naas.notification.send(email_to=email, subject=subject, html=html)

👌 💌 Email has been sent successfully !


### Automation

In [25]:
naas.dependency.add(markdown_path)
naas.dependency.add(excel_report)
#naas.scheduler.add(cron="30 13 * * *")

👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
